In [4]:
import xml.etree.ElementTree as ET
import datetime

In [10]:
alexStartTime = datetime.datetime.now()
alexRoot = ET.parse("alexAppleHealthData/export.xml").getroot()
alexEndTime = datetime.datetime.now()
print("Duration to capture Alex's ElementTree: %i sec" % ((alexEndTime-alexStartTime).seconds))
jamesStartTime = datetime.datetime.now()
jamesRoot = ET.parse("jamesAppleHealthData/export.xml").getroot()
jamesEndTime = datetime.datetime.now()
print("Duration to capture James' ElementTree: %i sec" % ((jamesEndTime-jamesStartTime).seconds))

Duration to capture Alex's ElementTree: 114 sec
Duration to capture James' ElementTree: 50 sec


In [24]:
def discoverRecordTypes(root, debug=0):
    typesDiscovered = []
    i = 0
    for child in root:
        i += 1
        if "type" not in child.attrib.keys():
            continue
        if i % 100000 == 0:
            if debug >= 1:
                print(i)
            if debug >= 2:
                print(typesDiscovered)
        if child.attrib["type"] not in typesDiscovered:
            if debug >= 2:
                print(child.attrib["type"])
            typesDiscovered.append(child.attrib["type"])
    return typesDiscovered

In [25]:
print("--Start Alex Type Captures--")
alexTypes = discoverRecordTypes(alexRoot, debug=1)
print("--End Alex Type Captures--")

print("--Start James Type Captures--")
jamesTypes = discoverRecordTypes(jamesRoot, debug=1)
print("--End James Type Captures--")

--Start Alex Type Captures--
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
--End Alex Type Captures--
--Start James Type Captures--
100000
200000
300000
400000
500000
600000
700000
800000
--End James Type Captures--


In [44]:
import csv
typeToTitleDict = {row[0]: row[1] for row in csv.reader(open("recordTypeToTitle.csv"))}

In [46]:
alexTitles = [typeToTitleDict[v] for v in alexTypes]
jamesTitles = [typeToTitleDict[v] for v in jamesTypes]

In [55]:
critType = 'HKQuantityTypeIdentifierActiveEnergyBurned'
for child in alexRoot:
    if "type" not in child.attrib.keys():
        continue
    if child.attrib['type'] == critType:
        alexChild = child
        break
for child in jamesRoot:
    if "type" not in child.attrib.keys():
        continue
    if child.attrib['type'] == critType:
        jamesChild = child
        break

In [56]:
jamesChild.attrib

{'creationDate': '2018-11-16 15:59:34 -0600',
 'device': '<<HKDevice: 0x2833d3520>, name:Apple Watch, manufacturer:Apple, model:Watch, hardware:Watch3,3, software:5.0>',
 'endDate': '2018-11-16 15:51:03 -0600',
 'sourceName': 'James’s Apple\xa0Watch',
 'sourceVersion': '5.0',
 'startDate': '2018-11-16 15:50:01 -0600',
 'type': 'HKQuantityTypeIdentifierActiveEnergyBurned',
 'unit': 'kcal',
 'value': '0.369'}

In [72]:
#AppleStandHour, AppleExerciseTime, ActiveEnergyBurned
def captureSingleRecordType(alexRoot, jamesRoot, typeToCapture):
    rtnList = [[], []]
    for idx, root in enumerate([alexRoot, jamesRoot]):
        for child in root:
            if "type" not in child.attrib.keys():
                continue
            if child.attrib['type'] == typeToCapture:
                rtnList[idx].append(child.attrib)
    return rtnList

In [73]:
rtnList = captureSingleRecordType(alexRoot, jamesRoot, "HKQuantityTypeIdentifierActiveEnergyBurned")

In [78]:
rtnList[0][0]

{'creationDate': '2019-04-25 18:21:44 -0600',
 'device': '<<HKDevice: 0x280a6f520>, name:Apple Watch, manufacturer:Apple, model:Watch, hardware:Watch1,1, software:4.3.2>',
 'endDate': '2019-04-25 18:20:39 -0600',
 'sourceName': 'Alex’s Apple\xa0Watch',
 'sourceVersion': '4.3.2',
 'startDate': '2019-04-25 18:19:38 -0600',
 'type': 'HKQuantityTypeIdentifierActiveEnergyBurned',
 'unit': 'kcal',
 'value': '0.408'}

In [82]:
import pandas as pd
df = pd.DataFrame(rtnList[0])
df.drop(labels=['device', 'sourceName', 'sourceVersion', 'type'], axis=1, inplace=True)
df.to_csv("alexEnergyData.csv")

In [83]:
df.head()

,creationDate,endDate,startDate,unit,value
0,2019-04-25 18:21:44 -0600,2019-04-25 18:20:39 -0600,2019-04-25 18:19:38 -0600,kcal,0.408
1,2019-04-25 18:21:44 -0600,2019-04-25 18:21:40 -0600,2019-04-25 18:20:39 -0600,kcal,0.354
2,2019-04-25 18:22:53 -0600,2019-04-25 18:22:42 -0600,2019-04-25 18:21:40 -0600,kcal,0.437
3,2019-04-25 18:24:54 -0600,2019-04-25 18:23:12 -0600,2019-04-25 18:22:42 -0600,kcal,0.204
4,2019-04-25 18:26:25 -0600,2019-04-25 18:24:13 -0600,2019-04-25 18:23:12 -0600,kcal,0.408


In [84]:
df.shape

(1375944, 5)